# Demo of `IdentityTransform`

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

plt.rcParams["figure.figsize"] = [10.5, 0.56 * 10.5]

In [ ]:
try:
    from bapsf_motion.transform import IdentityTransform
except (ModuleNotFoundError, ImportError):
    from pathlib import Path

    HERE = Path().cwd()
    BAPSF_MOTION = (HERE / ".." / ".." / ".." ).resolve()
    sys.path.append(str(BAPSF_MOTION))
    
    from bapsf_motion.transform import IdentityTransform

In [ ]:
tr = IdentityTransform(
    ("x", "y"),
)

In [ ]:
figwidth, figheight = plt.rcParams["figure.figsize"]
figwidth = 1.4 * figwidth
figheight = 2.0 * figheight
fig, axs = plt.subplots(2, 3, figsize=[figwidth, figheight])

axs[0,0].set_xlabel("MSpace X")
axs[0,0].set_ylabel("MSpace Y")
axs[0,1].set_xlabel("Drive X")
axs[0,1].set_ylabel("Drive Y")
axs[0,2].set_xlabel("MSpace X")
axs[0,2].set_ylabel("MSpace Y")

points = np.zeros((40, 2))
points[0:10, 0] = np.linspace(-5, 5, num=10, endpoint=False)
points[0:10, 1] = 5 * np.ones(10)
points[10:20, 0] = 5 * np.ones(10)
points[10:20, 1] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 0] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 1] = -5 * np.ones(10)
points[30:40, 0] = -5 * np.ones(10)
points[30:40, 1] = np.linspace(-5, 5, num=10, endpoint=False)

dpoints = tr(points, to_coords="drive")
mpoints = tr(dpoints, to_coords="motion_space")

axs[0,0].fill(points[...,0], points[...,1])
axs[0,1].fill(dpoints[...,0], dpoints[...,1])
axs[0,2].fill(mpoints[...,0], mpoints[...,1])

for pt, color in zip(
    [
        [-5, 5],
        [-5, -5],
        [5, -5],
        [5, 5],
        [0, 0]
    ],
    ["red", "orange", "green", "purple", "black"]
):
    dpt = tr(pt, to_coords="drive")
    mpt = tr(dpt, to_coords="motion_space")
    print(pt, dpt, mpt)
    axs[0,0].plot(pt[0], pt[1], 'o', color=color)
    axs[0,1].plot(dpt[..., 0], dpt[..., 1], 'o', color=color)
    axs[0,2].plot(mpt[..., 0], mpt[..., 1], 'o', color=color)

##

axs[1,0].set_xlabel("Drive X")
axs[1,0].set_ylabel("Drive Y")
axs[1,1].set_xlabel("MSpace X")
axs[1,1].set_ylabel("MSpace Y")
axs[1,2].set_xlabel("Drive X")
axs[1,2].set_ylabel("Drive Y")

points = np.zeros((40, 2))
points[0:10, 0] = np.linspace(-5, 5, num=10, endpoint=False)
points[0:10, 1] = 5 * np.ones(10)
points[10:20, 0] = 5 * np.ones(10)
points[10:20, 1] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 0] = np.linspace(5, -5, num=10, endpoint=False)
points[20:30, 1] = -5 * np.ones(10)
points[30:40, 0] = -5 * np.ones(10)
points[30:40, 1] = np.linspace(-5, 5, num=10, endpoint=False)

mpoints = tr(points, to_coords="motion_space")
dpoints = tr(mpoints, to_coords="drive")

axs[1,0].fill(points[...,0], points[...,1])
axs[1,1].fill(mpoints[...,0], mpoints[...,1])
axs[1,2].fill(dpoints[...,0], dpoints[...,1])

for pt, color in zip(
    [
        [-5, 5],
        [-5, -5],
        [5, -5],
        [5, 5],
        [0, 0]
    ],
    ["red", "orange", "green", "purple", "black"]
):
    mpt = tr(pt, to_coords="motion_space")
    dpt = tr(mpt, to_coords="drive")
    axs[1,0].plot(pt[0], pt[1], 'o', color=color)
    axs[1,1].plot(mpt[..., 0], mpt[..., 1], 'o', color=color)
    axs[1,2].plot(dpt[..., 0], dpt[..., 1], 'o', color=color)
    print(f"X = {pt[0]}  Δ = {dpt[...,0] - pt[0]} || Y = {pt[1]}  Δ = {dpt[...,1] - pt[1]}")


### Test Transforming `drive -> motion space -> drive`

In [ ]:
points = np.array([[5, 5], [5, 5]])
mpoints = tr(points, to_coords="motion_space")
dpoints = tr(mpoints, to_coords="drive")

(
    np.isclose(dpoints, points),
    np.allclose(dpoints, points),
    np.allclose(dpoints[...,0], points[...,0]),
    np.allclose(dpoints[...,1], points[...,1]),
    np.min(dpoints - points),
    np.max(dpoints - points),
)

### Test Transforming `motion space -> drive -> motion space`

In [ ]:
dpoints = tr(points, to_coords="drive")
mpoints = tr(dpoints, to_coords="motion_space")

(
    np.allclose(mpoints, points),
    np.allclose(mpoints[...,0], points[...,0]),
    np.allclose(mpoints[...,1], points[...,1]),
    np.min(mpoints - points),
    np.max(mpoints - points),
)

## Prototyping

In [ ]:
pts = [
    [-5, 5],
    [-5, -5],
    [5, -5],
    [5, 5],
    [0, 0]
]
# pts = [[-5, 5]]

pts = tr._condition_points(pts)
matrix = tr.matrix(pts, to_coords="mspace")
pts = np.concatenate(
    (pts, np.ones((pts.shape[0], 1))),
    axis=1,
)
results = np.einsum("kmn,kn->km", matrix, pts)[:-1,...]
ii = 1
# pts[ii, ...]
(pts[ii,...], results[ii,...])

In [ ]:
matrix[ii, ...]

In [ ]:
(
    pts[ii, :-1],
    tr(pts[ii, :-1], to_coords="mspace"),
)

In [ ]:
tr(pts[ii, :-1], to_coords="mspace")